# Simulated Scan Strategies

In this lesson we learn how to simulate satellite and ground observing schedules

In [ ]:
# Are you using a special reservation for a workshop?
# If so, set it here:
nersc_reservation = None

# Load common tools for all lessons
import sys
sys.path.insert(0, "..")
from lesson_tools import (
    check_nersc,
    fake_focalplane
)
nersc_host, nersc_repo = check_nersc(reservation=nersc_reservation)
if nersc_host is not None:
    %reload_ext slurm_magic

## Running in Parallel

The NERSC login nodes do not support MPI, so all of the previous examples are running serially.  To run in parallel, we can submit a batch job version of the above examples:

In [ ]:
%%writefile simscan.py

import toast
from toast.mpi import MPI


In [ ]:
if nersc_host is not None:
    %srun -N 1 -C knl -n 32 -c 2 --cpu_bind=cores -t 00:03:00 python simscan.py

## Ground observing schedules

TOAST `pipelines` include a tool called `toast_ground_schedule.py`, also known as the *opportunistic scheduler*. It can be used to create site-specific observing schedules subject to a number of constraints. At minimum, the tool needs the location of the observatory, observing time and at least one target. Here is a minimal example:

In [ ]:
! toast_ground_schedule.py \
    --site-lat "-22.958064" \
    --site-lon "-67.786222" \
    --site-alt 5200 \
    --site-name Atacama \
    --telescope LAT \
    --start "2020-01-01 00:00:00" \
    --stop "2020-01-01 12:00:00" \
    --patch-coord C \
    --patch small_patch,1,40,-40,44,-44 \
    --out schedule.txt

! cat schedule.txt

The rectangular patch definition takes the form `--patch <name>,<priority>,<RA left>,<DEC top>,<RA right>,<DEC bottom>`. No spaces are allowed in the definition.  Other patch definition formats will be discussed below.

The resulting schedule is a plain ASCII file. The header defines the telescope and each line after that defines a constant elevation scan (CES) with a fixed azimuth range. When a full pass of the target takes longer than allowed observation time, `--ces-max-time`, the CES is broken up into sub passes that use the same observing elevation but adjust the azimuth range.  The above schedule includes 10 passes of the target "small_patch" that fit in the allowed 12-hour observing time. Some passes are split into as many as 4 sub passes, each no longer than 20 minutes (default).

Let's add another patch using the circular patch definition format, set the observing elevation limits and enable Sun avoidance. We'll also increase `ces-max-time` So we get fewer entries in the schedule.

In [ ]:
! toast_ground_schedule.py \
    --site-lat "-22.958064" \
    --site-lon "-67.786222" \
    --site-alt 5200 \
    --site-name Atacama \
    --telescope LAT \
    --start "2020-01-01 00:00:00" \
    --stop "2020-01-01 12:00:00" \
    --patch-coord C \
    --patch small_patch,1,40,-40,44,-44 \
    --patch large_patch,1,20.,-42.,10. \
    --el-min 40 \
    --el-max 60 \
    --ces-max-time 86400 \
    --sun-avoidance-angle 20 \
    --out schedule.txt

! cat schedule.txt

In [ ]:
! python3 -c "from toast.mpi import MPI; print(MPI)"